In [5]:
class TreeNode:
    
    # function: _init_
    # @param:
    #     item: itemset
    #     count: number of items
    #     parent: parent node
    #     children: list of children, init with {}
    #     nodeLink: node link to other sample node, init with None
    def __init__(self, item, count, parent):
        self.item = item       
        self.count = count     
        self.parent = parent   
        self.children = {}     
        self.nodeLink = None   

    # function: increment (update count value)
    # @param:
    #     count: update element -> self.count+=count
    def increment(self, count):
        # 更新计数
        self.count += count

    # function: display (print tree item and count)
    # @param:
    #     ind=1
    # note: child in self.children.values() recursion(ind+1)
    def display(self, ind=1):
        print('  ' * ind, self.item, ' ', self.count)
        for child in self.children.values():
            child.display(ind + 1)


# function: createTree(dataSet,minSup)
# @param: 
#       dataSet: retail transections
#       minSup: min_support
def createTree(dataSet, minSup):
    headerTable = {}
    # loop1: trans in dataSet -> each transaction set
    # loop2: item in trans -> each transection 
    # function: headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]

    # k: key
    # headerTable[k]: value
    # if value < minSup then del this value
    for k in list(headerTable):
        if headerTable[k] < minSup:
            del (headerTable[k])
            
    # Init freqItemSet to set(headerTable.keys())
    freqItemSet = set(headerTable.keys())

    # return None None if length is 0
    if len(freqItemSet) == 0:
        return None, None

    # map all keys to a list [count, node]
    # headerTable[k] -> [headerTable[k], None]
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]

    # retTree: root, init with 'TreeNode('Null Set', 1, None)'
    retTree = TreeNode('Null Set', 1, None)

    # loop1: tranSet and count in dataSet.items()
    # init a localD -> {}
    # loop2: item freqItemSet
    # if item in FreqItemset then localD[item] -> headerTable[item][0]
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            # generate orderedItems -> [v[0] for v in sorted(localD.items(), key=lambda p: (-p[1], p[0]))]
            # update Tree with sorted itemsets
            # updateTree(orderedItems, retTree, headerTable, count)
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: (-p[1], p[0]))]
            updateTree(orderedItems, retTree, headerTable, count)

    return retTree, headerTable


def updateTree(items, inTree, headerTable, count):
    # items[0] in inTree.children then inTree.children[items[0]].increment(count)
    if items[0] in inTree.children:
        inTree.children[items[0]].increment(count)
    else:
        # if not exist then create a new TreeNode(items[0], count, inTree) to inTree.children[items[0]]
        inTree.children[items[0]] = TreeNode(items[0], count, inTree)
        # create a new link if never appeered before headerTable[items[0]][1] = inTree.children[items[0]]
        # else: update
        if headerTable[items[0]][1] is None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # if len(items)>1 then recursion with updateTree(items[1::], inTree.children[items[0]], headerTable, count)
    if(len(items)>1):
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)


def updateHeader(nodeToTest, targetNode):
    # loop while: nodeToTest.nodeLink not None then nodeToTest = nodeToTest.nodeLink
    while nodeToTest.nodeLink is not None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode


def ascendTree(leafNode, prefixPath):
    # if leafNode.parent is not None then prefix.append(leafNode.item) and recursion(leafNode.parent, prefixPath)
    if leafNode.parent is not None:
        prefixPath.append(leafNode.item)
        ascendTree(leafNode.parent, prefixPath)


# @return: Conditional Pattern Base condPats {}
def findPrefixPath(basePat, treeNode):
    # init with a dict condPats
    condPats = {}
    # loop condition: until treeNode is None
    while treeNode is not None:
        # store prefixPath with a list init to []
        # Obtain the prefix path ending with basePat ascendTree(treeNode, prefixPath)
        prefixPath = []
        ascendTree(treeNode, prefixPath)  
        # if prefixPath length >1 then {prefixPath[1:],treeNode.count}
        # after judge update treeNode->treeNode.nodeLink
        if (len(prefixPath)) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats


# Exploit frequency itemsets
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # all frequency items bigL ->  [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]
    # For each frequent itemset, add it to preFix and output it
    for basePat in bigL: 
        # copy preFix and add basePat to newFreqSet
        # append
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        # Obtain all prefix paths ending with this item and create a conditional FP tree
        # condPattBases -> findPrefixPath(basePat, headerTable[basePat][1])
        # create  myCondTree, myHead -> createTree(condPattBases, minSup)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)

        # If there is a frequent itemset in the conditional FP tree, recursively call the mineTree function
        # myHead not None then mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
        if myHead is not None:
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)


def loadData():
    data = []

    with open('retail.dat') as f:
        for line in f:
            items = line.strip().split()
            data.append(frozenset(items))

    return data


minSup = 0.0005*88163
dataSet = {}

transections = loadData()
print(len(transections))

for trans in transections:
    dataSet[trans] = dataSet.get(trans, 0) + 1

myFPtree, myHeaderTab = createTree(dataSet, minSup)
freqItems = []
mineTree(myFPtree, myHeaderTab, minSup, set([]), freqItems)

print("Frequent Items:")
dic_res={}
total=0
for itemset in freqItems:
    dic_res[len(itemset)]=dic_res.get(len(itemset),0)+1

for key,value in dic_res.items():
    print(key,end=' ')
    print(value)
print(sum(dic_res.values()))


88162
Frequent Items:
1 3926
2 8198
3 5411
4 1534
5 170
6 3
19242
